# Walmart Recruiting: Trip Type Classification
Classification Problem, try XGBoost.

## Import important libraries

In [1]:
import pandas as pd
import numpy as np

## Import dataset

In [2]:
dataset_import = pd.read_csv('/Users/christophodermatt/git/kaggle/train.csv')

In [3]:
# Check the first couple of rows
dataset_import.head(10)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,6.811315e+10,-1,FINANCIAL SERVICES,1000.0
1,30,7,Friday,6.053882e+10,1,SHOES,8931.0
2,30,7,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
3,26,8,Friday,2.238404e+09,2,PAINT AND ACCESSORIES,3565.0
4,26,8,Friday,2.006614e+09,2,PAINT AND ACCESSORIES,1017.0
5,26,8,Friday,2.006619e+09,2,PAINT AND ACCESSORIES,1017.0
6,26,8,Friday,2.006614e+09,1,PAINT AND ACCESSORIES,1017.0
7,26,8,Friday,7.004803e+09,1,PAINT AND ACCESSORIES,2802.0
8,26,8,Friday,2.238495e+09,1,PAINT AND ACCESSORIES,4501.0
9,26,8,Friday,2.238400e+09,-1,PAINT AND ACCESSORIES,3565.0


## Data transformation

In [4]:
# Check the number of data points in the dataset
print(len(dataset_import))
# Check the number of features in the dataset
print(len(dataset_import.columns))
# Check the data types
print(dataset_import.dtypes.unique())

647054
7
[dtype('int64') dtype('O') dtype('float64')]


In [5]:
# Check the object type columns
object_type_list = dataset_import.select_dtypes(include = ['O']).columns.tolist()
print(object_type_list)

['Weekday', 'DepartmentDescription']


In [6]:
# Check the features with categories
for name in object_type_list:
    print(dataset_import.loc[:, name].value_counts())

Sunday       133975
Saturday     122096
Friday        96247
Monday        83130
Tuesday       72529
Wednesday     71115
Thursday      67962
Name: Weekday, dtype: int64
GROCERY DRY GOODS              70402
DSD GROCERY                    68332
PRODUCE                        51115
DAIRY                          43820
PERSONAL CARE                  41969
IMPULSE MERCHANDISE            28712
HOUSEHOLD CHEMICALS/SUPP       24880
PHARMACY OTC                   23306
FROZEN FOODS                   21101
HOUSEHOLD PAPER GOODS          16274
COMM BREAD                     15244
BEAUTY                         14865
MENS WEAR                      12250
FINANCIAL SERVICES             10723
INFANT CONSUMABLE HARDLINES    10646
SERVICE DELI                   10127
CANDY, TOBACCO, COOKIES         9808
PETS AND SUPPLIES               9761
MEAT - FRESH & FROZEN           9477
PRE PACKED DELI                 9052
LADIESWEAR                      8721
CELEBRATION                     8698
COOK AND DINE     

In [7]:
# Check any number of columns with NaN
print(dataset_import.isnull().any().sum(), ' / ', len(dataset_import.columns))
# Check any number of data points with NaN
print(dataset_import.isnull().any(axis = 1).sum(), 
    ' / ', 
    len(dataset_import), 
    ' / {0:.4f}%'.format(dataset_import.isnull().any(axis = 1).sum() / len(dataset_import) * 100))

3  /  7
4129  /  647054  / 0.6381%


In [8]:
# Check which columns have NaN values
null_data_columns = []
for number, name in enumerate(dataset_import.isnull().any()):
    if name == True:
        null_data_columns.append(dataset_import.isnull().any().index[number])
null_data_columns

['Upc', 'DepartmentDescription', 'FinelineNumber']

In [9]:
# Check some of those rows
null_data = dataset_import[dataset_import.isnull().any(axis = 1)]
null_data.head(20)

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
25,26,8,Friday,NaN,1,NaN,NaN
548,27,259,Friday,NaN,3,NaN,NaN
549,27,259,Friday,NaN,1,NaN,NaN
959,999,409,Friday,NaN,-1,NaN,NaN
1116,39,479,Friday,NaN,1,NaN,NaN
1134,999,484,Friday,NaN,-2,NaN,NaN
1135,999,484,Friday,NaN,-2,NaN,NaN
1155,44,496,Friday,NaN,1,PHARMACY RX,NaN
1216,5,521,Friday,NaN,1,PHARMACY RX,NaN
1373,5,585,Friday,NaN,1,PHARMACY RX,NaN


In [10]:
# Looks like Upc and FinelineNumber are all NaN, sometimes DepartmentDescription
for number, _ in enumerate(null_data_columns):
    print(null_data.loc[:, null_data_columns[number]].isnull().sum())

4129
1361
4129


In [11]:
# Upc and FinelineNumber are not easy to replace since they are product IDs.
# Also, 4129 is only 0.64% of the dataset, so I drop them from my initial research.
# So we could use either Upc, DepartmentDescription or FinelineNumber.
# For this run, I am going to use DepartmentDescription, 
# the other two categories may be better since they are more detailed. They could be used later.

In [12]:
# Drop Upc and FinelineNumber and use DepartmentDescritpion as a dummies.
# Hence no need to drop all NaN's but rather the NaN's in DepartmentDescription.
walmart_train_reduced = dataset_import.iloc[:, [0, 1, 2, 4, 5]]
walmart_train_reduced = walmart_train_reduced.dropna()
walmart_train_reduced.head(20)

,TripType,VisitNumber,Weekday,ScanCount,DepartmentDescription
0,999,5,Friday,-1,FINANCIAL SERVICES
1,30,7,Friday,1,SHOES
2,30,7,Friday,1,PERSONAL CARE
3,26,8,Friday,2,PAINT AND ACCESSORIES
4,26,8,Friday,2,PAINT AND ACCESSORIES
5,26,8,Friday,2,PAINT AND ACCESSORIES
6,26,8,Friday,1,PAINT AND ACCESSORIES
7,26,8,Friday,1,PAINT AND ACCESSORIES
8,26,8,Friday,1,PAINT AND ACCESSORIES
9,26,8,Friday,-1,PAINT AND ACCESSORIES


In [13]:
# Check for misspellings
new_features = sorted(walmart_train_reduced.DepartmentDescription.unique())
print(new_features)
print(len(new_features))

['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES', 'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES', 'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD', 'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY', 'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES', 'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14', 'GROCERY DRY GOODS', 'HARDWARE', 'HEALTH AND BEAUTY AIDS', 'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL', 'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES', 'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN', 'MEDIA AND GAMING', 'MENS WEAR', 'MENSWEAR', 'OFFICE SUPPLIES', 'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS', 'PAINT AND ACCESSORIES', 'PERSONAL CARE', 'PETS AND SUPPLIES', 'PHARMACY OTC', 

In [14]:
# MENS WEAR and MENSWEAR are misspelled
walmart_train_reduced.loc[
    :, 
    'DepartmentDescription'
    ] = walmart_train_reduced.loc[
        :, 
        'DepartmentDescription'
        ].str.replace(
            'MENSWEAR', 
            'MENS WEAR'
            )

In [15]:
# Check if it worked
print(list(sorted(walmart_train_reduced.DepartmentDescription.unique())))
print(len(list(sorted(walmart_train_reduced.DepartmentDescription.unique()))))

['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES', 'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES', 'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD', 'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY', 'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES', 'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14', 'GROCERY DRY GOODS', 'HARDWARE', 'HEALTH AND BEAUTY AIDS', 'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL', 'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES', 'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN', 'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES', 'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS', 'PAINT AND ACCESSORIES', 'PERSONAL CARE', 'PETS AND SUPPLIES', 'PHARMACY OTC', 'PHARMACY RX

In [16]:
# Use DepartmentDescription as features, with the ScanCount as values and sum them to one row per single visit
dummy_features = pd.get_dummies(walmart_train_reduced.iloc[:, 4])
dummy_features = dummy_features.mul(walmart_train_reduced['ScanCount'], axis=0)
dummy_features['TripType'] = walmart_train_reduced['TripType']
dummy_features['VisitNumber'] = walmart_train_reduced['VisitNumber']
dummy_features['Weekday'] = walmart_train_reduced['Weekday']
walmart_train_wide = dummy_features.groupby(['TripType', 'VisitNumber', 'Weekday']).sum().reset_index()

In [17]:
walmart_train_wide.head(20)

,TripType,VisitNumber,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,3,106,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,121,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,153,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,162,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,164,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,3,177,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,3,181,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,3,188,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,3,203,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,3,265,Friday,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Check shape of the dataset
walmart_train_wide.shape

(95516, 70)

In [19]:
# Slice and convert the data into numpy arrays
X = walmart_train_wide.iloc[:, 2:].values
y = walmart_train_wide.iloc[:, 0].values

In [20]:
# Check the resulting matrix
X[:5]

array([['Friday', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 

In [21]:
# Encoding categorical independent variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [22]:
# Check the new shape and matrix
np.shape(X), X[:10, :9], X[-10:, :9]

((95516, 73), array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), array([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0

# XGBoost

In [23]:
#Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 9444)

In [25]:
# Fitting the Classification Model to the dataset using some arbitrary parameters
from xgboost import XGBClassifier
xgb_classifier = XGBClassifier(
    n_estimators = 100, 
    learning_rate = 0.08, 
    gamma=0.01,
    subsample = 0.75, 
    colsample_bytree = 1, 
    max_depth = 7, 
    n_jobs = -1
    )
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.01, learning_rate=0.08,
       max_delta_step=0, max_depth=7, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=-1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.75)

In [26]:
# Predicting the test set results
y_pred = xgb_classifier.predict(X_test)

In [27]:
# Evaluate our predictions
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {0:.2f}%".format(accuracy * 100))

Accuracy: 63.87%


In [28]:
# Check our output and compare
y_pred[:50], y_test[:50]

(array([  9,   8,  15,   8,   8,  40,   5,  15,  24,   8, 999,   9,  25,
        999,  37,  38,   8,  27,   8,   7,  32,  28,   9,  42,  35,   8,
         34,  36,   5,   6,   7,  33,  39,  24,  42,   7,   5,  35,   8,
          3,   6,  36,  36,   8,   9,   5,  25, 999,  35,   3]),
 array([  9,   8,  15,  37,   8,  40,   5,  15,  24,   8, 999,   7,  42,
        999,  37,  38,   8,  27,   7,  36,  32,  28, 999,  41,   6,   8,
         41,   8,   4,  38,   8,  33,  24,  24,  44,   7,   5,  29,   8,
        999,   6,  36,  39,   8, 999,  12,  25, 999,   8,   3]))

## Run with Kaggle test set

In [29]:
# Use test csv
dataset_import = pd.read_csv('/Users/christophodermatt/git/kaggle/test.csv')

In [30]:
# Check the first couple of rows
dataset_import.head(10)

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,7.250339e+10,1,SHOES,3002.0
1,1,Friday,1.707711e+09,1,DAIRY,1526.0
2,1,Friday,8.947000e+10,1,DAIRY,1431.0
3,1,Friday,8.849121e+10,1,GROCERY DRY GOODS,3555.0
4,2,Friday,2.840015e+09,1,DSD GROCERY,4408.0
5,2,Friday,7.874205e+09,1,BAKERY,5019.0
6,2,Friday,8.745860e+10,1,IMPULSE MERCHANDISE,8023.0
7,2,Friday,8.745860e+10,1,IMPULSE MERCHANDISE,8023.0
8,3,Friday,7.410811e+09,1,PERSONAL CARE,4504.0
9,3,Friday,7.410811e+09,-1,PERSONAL CARE,4504.0


## Data transformation

In [31]:
# Check the number of data points in the data set
print(len(dataset_import))
# Check the number of features in the data set
print(len(dataset_import.columns))
# Check the data types
print(dataset_import.dtypes.unique())

653646
6
[dtype('int64') dtype('O') dtype('float64')]


In [32]:
# Check the object type columns
object_type_list = dataset_import.select_dtypes(include = ['O']).columns.tolist()
print(object_type_list)

['Weekday', 'DepartmentDescription']


In [33]:
# Check the features with categories
for name in object_type_list:
    print(dataset_import.loc[:, name].value_counts())

Sunday       133233
Saturday     123974
Friday        94764
Monday        85174
Tuesday       73742
Wednesday     72481
Thursday      70278
Name: Weekday, dtype: int64
GROCERY DRY GOODS              72335
DSD GROCERY                    68860
PRODUCE                        50928
DAIRY                          44824
PERSONAL CARE                  41607
IMPULSE MERCHANDISE            28441
HOUSEHOLD CHEMICALS/SUPP       24602
PHARMACY OTC                   23293
FROZEN FOODS                   21890
COMM BREAD                     15688
HOUSEHOLD PAPER GOODS          15642
BEAUTY                         15223
MENS WEAR                      12171
FINANCIAL SERVICES             10857
INFANT CONSUMABLE HARDLINES    10670
CANDY, TOBACCO, COOKIES        10310
SERVICE DELI                   10049
MEAT - FRESH & FROZEN           9936
PETS AND SUPPLIES               9694
PRE PACKED DELI                 9276
LADIESWEAR                      9019
CELEBRATION                     8635
COOK AND DINE     

In [34]:
# Check any number of columns with NaN
print(dataset_import.isnull().any().sum(), ' / ', len(dataset_import.columns))
# Check any number of data points with NaN
print(dataset_import.isnull().any(axis = 1).sum(), 
    ' / ', 
    len(dataset_import), 
    ' / {0:.4f}%'.format(dataset_import.isnull().any(axis = 1).sum() / len(dataset_import) * 100))

3  /  6
3986  /  653646  / 0.6098%


In [35]:
# Check which columns have NaN values
null_data_columns = []
for number, name in enumerate(dataset_import.isnull().any()):
    if name == True:
        null_data_columns.append(dataset_import.isnull().any().index[number])
null_data_columns

['Upc', 'DepartmentDescription', 'FinelineNumber']

In [36]:
# Check some of those rows
null_data = dataset_import[dataset_import.isnull().any(axis = 1)]
null_data.head(20)

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
621,254,Friday,NaN,1,NaN,NaN
1188,512,Friday,NaN,1,PHARMACY RX,NaN
1189,512,Friday,NaN,1,PHARMACY RX,NaN
1190,512,Friday,NaN,1,PHARMACY RX,NaN
1314,561,Friday,NaN,1,PHARMACY RX,NaN
1315,561,Friday,NaN,1,PHARMACY RX,NaN
1351,590,Friday,NaN,1,NaN,NaN
1845,777,Friday,NaN,1,PHARMACY RX,NaN
1945,810,Friday,NaN,1,PHARMACY RX,NaN
1946,810,Friday,NaN,1,PHARMACY RX,NaN


In [37]:
# Looks like Upc and FinelineNumber are all NaN, sometimes DepartmentDescription
for number, _ in enumerate(null_data_columns):
    print(null_data.loc[:, null_data_columns[number]].isnull().sum())

3986
1328
3986


In [38]:
# Drop Upc and FinelineNumber and use DepartmentDescritpion as a dummies
# Hence no need to drop all NaN's but rather the NaN's in DepartmentDescription
walmart_test_reduced = dataset_import.iloc[:, [0, 1, 3, 4]]
walmart_test_reduced = walmart_test_reduced.dropna()
walmart_test_reduced.head(20)

,VisitNumber,Weekday,ScanCount,DepartmentDescription
0,1,Friday,1,SHOES
1,1,Friday,1,DAIRY
2,1,Friday,1,DAIRY
3,1,Friday,1,GROCERY DRY GOODS
4,2,Friday,1,DSD GROCERY
5,2,Friday,1,BAKERY
6,2,Friday,1,IMPULSE MERCHANDISE
7,2,Friday,1,IMPULSE MERCHANDISE
8,3,Friday,1,PERSONAL CARE
9,3,Friday,-1,PERSONAL CARE


In [39]:
walmart_test_reduced.shape

(652318, 4)

In [40]:
# Check for misspellings
new_features = sorted(walmart_test_reduced.DepartmentDescription.unique())
print(new_features)
print(len(new_features))

['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES', 'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES', 'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD', 'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY', 'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES', 'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14', 'GROCERY DRY GOODS', 'HARDWARE', 'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL', 'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES', 'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN', 'MEDIA AND GAMING', 'MENS WEAR', 'MENSWEAR', 'OFFICE SUPPLIES', 'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS', 'PAINT AND ACCESSORIES', 'PERSONAL CARE', 'PETS AND SUPPLIES', 'PHARMACY OTC', 'PHARMACY RX', 'PLAYERS AN

In [41]:
# MENS WEAR and MENSWEAR are misspelled
walmart_test_reduced.loc[
    :, 
    'DepartmentDescription'
    ] = walmart_test_reduced.loc[
        :, 
        'DepartmentDescription'
        ].str.replace(
            'MENSWEAR', 
            'MENS WEAR'
            )

In [42]:
# Check if it worked
print(list(sorted(walmart_test_reduced.DepartmentDescription.unique())))
print(len(list(sorted(walmart_test_reduced.DepartmentDescription.unique()))))

['1-HR PHOTO', 'ACCESSORIES', 'AUTOMOTIVE', 'BAKERY', 'BATH AND SHOWER', 'BEAUTY', 'BEDDING', 'BOOKS AND MAGAZINES', 'BOYS WEAR', 'BRAS & SHAPEWEAR', 'CAMERAS AND SUPPLIES', 'CANDY, TOBACCO, COOKIES', 'CELEBRATION', 'COMM BREAD', 'CONCEPT STORES', 'COOK AND DINE', 'DAIRY', 'DSD GROCERY', 'ELECTRONICS', 'FABRICS AND CRAFTS', 'FINANCIAL SERVICES', 'FROZEN FOODS', 'FURNITURE', 'GIRLS WEAR, 4-6X  AND 7-14', 'GROCERY DRY GOODS', 'HARDWARE', 'HOME DECOR', 'HOME MANAGEMENT', 'HORTICULTURE AND ACCESS', 'HOUSEHOLD CHEMICALS/SUPP', 'HOUSEHOLD PAPER GOODS', 'IMPULSE MERCHANDISE', 'INFANT APPAREL', 'INFANT CONSUMABLE HARDLINES', 'JEWELRY AND SUNGLASSES', 'LADIES SOCKS', 'LADIESWEAR', 'LARGE HOUSEHOLD GOODS', 'LAWN AND GARDEN', 'LIQUOR,WINE,BEER', 'MEAT - FRESH & FROZEN', 'MEDIA AND GAMING', 'MENS WEAR', 'OFFICE SUPPLIES', 'OPTICAL - FRAMES', 'OPTICAL - LENSES', 'OTHER DEPARTMENTS', 'PAINT AND ACCESSORIES', 'PERSONAL CARE', 'PETS AND SUPPLIES', 'PHARMACY OTC', 'PHARMACY RX', 'PLAYERS AND ELECTRONIC

In [43]:
# Check if the features are the same
missing_feature = set(sorted(
    walmart_train_reduced.DepartmentDescription.unique()
    )) - set(sorted(
        walmart_test_reduced.DepartmentDescription.unique()
        ))
missing_feature = tuple(missing_feature)
missing_feature

('HEALTH AND BEAUTY AIDS',)

In [44]:
# Use DepartmentDescription as features, with the ScanCount as values and sum them to one row per single visit
# Add HEALTH AND BEAUTY AIDS to test set
dummy_features = pd.get_dummies(walmart_test_reduced.iloc[:, 3])
dummy_features = dummy_features.mul(walmart_test_reduced['ScanCount'], axis=0)
dummy_features.insert(loc = 0, column = missing_feature, value = np.zeros(len(dummy_features), dtype=np.int))
dummy_features = dummy_features[list(sorted(walmart_train_reduced.DepartmentDescription.unique()))]
dummy_features['VisitNumber'] = walmart_test_reduced['VisitNumber']
dummy_features['Weekday'] = walmart_test_reduced['Weekday']
walmart_test_wide = dummy_features.groupby(['VisitNumber', 'Weekday']).sum().reset_index()

In [45]:
walmart_test_wide.head(20)

,VisitNumber,Weekday,1-HR PHOTO,ACCESSORIES,AUTOMOTIVE,BAKERY,BATH AND SHOWER,BEAUTY,BEDDING,BOOKS AND MAGAZINES,...,SEAFOOD,SEASONAL,SERVICE DELI,SHEER HOSIERY,SHOES,SLEEPWEAR/FOUNDATIONS,SPORTING GOODS,SWIMWEAR/OUTERWEAR,TOYS,WIRELESS
0,1,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,2,Friday,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,13,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,14,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,16,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,18,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,21,Friday,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
# Check shape of the dataset
walmart_test_wide.shape

(95496, 69)

In [47]:
# Slice and convert the data into numpy arrays
X = walmart_test_wide.iloc[:, 1:].values

In [48]:
# Check the resulting matrix
X[:5]

array([['Friday', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0],
       ['Friday', 0, 0, 

In [49]:
# Encoding categorical independent variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X = LabelEncoder()
X[:, 0] = labelencoder_X.fit_transform(X[:, 0])
onehotencoder = OneHotEncoder(categorical_features = [0])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [50]:
# Check the new shape and matrix
np.shape(X), X[:10, :9], X[-10:, :9]

((95496, 73), array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), array([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0

# XGBoost

In [51]:
# Predicting the Kaggle test set results
y_pred_kaggle = xgb_classifier.predict(X)

In [52]:
y_pred_kaggle[:50]

array([ 30,   8,   9,   9,   9,   7,  39,  35,  39,   5,   8,   6,  35,
         8,   6,   8,   8,   8,   8,   8,   6,  39,   9,   9,   9,   8,
         8,   8,   9,  40,   9,  25,   5,   9,   9,  34,   6,  39,   8,
        39,   7, 999,   5,   3,  35,   6,   7,   8,  31,   8])

In [53]:
# Put results in the asked format to test against the Kaggle test dataset
results = pd.get_dummies(y_pred_kaggle)

In [54]:
# Check results
results.head(20)

,3,5,6,7,8,9,12,15,18,19,...,36,37,38,39,40,41,42,43,44,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
9,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# Check the format of our results, if columns are missing
print(results.columns.tolist())
print(len(results.columns.tolist()))
# Compare against original list
print(sorted(walmart_train_reduced.loc[:, 'TripType'].unique()))
print(len(walmart_train_reduced.loc[:, 'TripType'].unique()))

[3, 5, 6, 7, 8, 9, 12, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 999]
36
[3, 4, 5, 6, 7, 8, 9, 12, 14, 15, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 999]
38


In [56]:
# Two features are missing, TripTypes 4 and 14. Add them to results with all 0's
results.insert(loc = 0, column = 4, value = np.zeros(len(results), dtype=np.int))
results.insert(loc = 0, column = 14, value = np.zeros(len(results), dtype=np.int))
# sort to match Kaggle format
kaggle_results = results.sort_index(axis=1)
kaggle_results.index = walmart_test_wide['VisitNumber']

In [57]:
# Check transformation
kaggle_results.head(20)

,3,4,5,6,7,8,9,12,14,15,...,36,37,38,39,40,41,42,43,44,999
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [58]:
# Change the header names, if important
new_headers = []
for header in kaggle_results.columns:
    new_headers.append('TripType_{}'.format(header))
kaggle_results.columns = new_headers

In [59]:
# Check transformation
kaggle_results.head(10)

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [60]:
# Append the dropped data
nan_visit_number = null_data[null_data['DepartmentDescription'].isnull() == True]
append_dataset = pd.DataFrame(0, index = nan_visit_number['VisitNumber'].unique(), columns = kaggle_results.columns)
kaggle_results = kaggle_results.append(append_dataset, ignore_index = False).sort_index()
kaggle_results = kaggle_results.groupby(kaggle_results.index).sum()

In [61]:
# Check if VisitNumber 254 was added, and at the right place
kaggle_results.iloc[125:135]

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
246,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
247,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
262,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
264,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
# Check shape for submission
kaggle_results.shape[0] - 95674

0

In [63]:
kaggle_results.to_csv('/Users/christophodermatt/git/kaggle/results.csv')

In [ ]:
'''
Add VisitNumber in cell A1.
First submission result with n_estimators = 50 was 13.28,
second submission result with n_estimators = 100 was 12.63.
'''

In [ ]:
'''
Other things that could have been done (more features).
- adding a column of total numbers of goods in that visit;
- using Upc or FinelineNumbers as features instead of DepartmentDescription;
- tune parameters (see below), needs computing time, or do on Google's Datalab
'''

In [ ]:
# A parameter grid for XGBoost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
cv_params = {
    'min_child_weight': [1, 3, 5], 
    'gamma': [0.5, 1, 2, 3],  
    'subsample': [i / 10.0 for i in range(6, 11)],
    'colsample_bytree': [i / 10.0 for i in range(6, 11)], 
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.02, 0.1]
    }

# Initialize XGB and GridSearch
xgb_for_gridsearch = XGBClassifier(
    n_estimators = 1000, 
    seed = 9444
    ) 

xgb_grid = GridSearchCV(
    estimator = xgb_for_gridsearch, 
    param_grid = cv_params,
    cv = 3, 
    n_jobs = -1
    )

xgb_grid.fit(X_train, y_train)

In [ ]:
# Show all the results
xgb_grid.cv_results_

In [ ]:
# Show best parameters
print(xgb_grid.best_params_)
# Show best estimator
print(xgb_grid.best_estimator_)
# Show best score
print(xgb_grid.best_score_)